In [15]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function,
)
#from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

In [ ]:
config_list = [
    {
        
    }
]
llm_config = {
    "timeout": 600,
    "cache_seed": 42,  # Use 'cache_seed' instead of 'use_cache' (True becomes a seed value)
    "config_list": config_list,
}

In [4]:
task = (
    f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Nvidia in the past month."
)
print(task)

Today is 2025-05-07. Write a blogpost about the stock price performance of Nvidia in the past month.


## Approach 1. Two-agent chat with function call for task decomposition

In [5]:
planner = AssistantAgent(
    name="planner",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
    system_message="You are a helpful AI assistant. You suggest a feasible plan "
    "for finishing a complex task by decomposing it into 3-5 sub-tasks. "
    "If the plan is not good, suggest a better plan. "
    "If the execution is wrong, analyze the error and suggest a fix.",
)

In [6]:
# Create a planner user agent used to interact with the planner.
planner_user = UserProxyAgent(
    name="planner_user",
    human_input_mode="NEVER",
    code_execution_config=False,
)

In [7]:
def task_planner(question: Annotated[str, "Question to ask the planner."]) -> str:
    with Cache.disk(cache_seed=4) as cache:
        planner_user.initiate_chat(planner, message=question, max_turns=1, cache=cache)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

In [8]:
# Create assistant agent.
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful AI assistant. "
    "You can use the task planner to decompose a complex task into sub-tasks. "
    "Make sure your follow through the sub-tasks. "
    "When needed, write Python code in markdown blocks, and I will execute them."
    "Give the user a final solution at the end. "
    "Return TERMINATE only if the sub-tasks are completed.",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
)

In [9]:
os.makedirs("planning", exist_ok=True)

In [10]:
# code_executor = DockerCommandLineCodeExecutor(work_dir="planning")
code_executor = LocalCommandLineCodeExecutor(work_dir="planning")

In [11]:
# Create user proxy agent used to interact with the assistant.
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"executor": code_executor},
)

In [12]:
# Register the function to the agent pair.
register_function(
    task_planner,
    caller=assistant,
    executor=user_proxy,
    name="task_planner",
    description="A task planner than can help you with decomposing a complex task into sub-tasks.",
)

In [13]:
# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with Cache.disk(cache_seed=1) as cache:
    # the assistant receives a message from the user, which contains the task description
    user_proxy.initiate_chat(
        assistant,
        message=task,
        cache=cache,
    )

user_proxy (to assistant):

Today is 2025-05-07. Write a blogpost about the stock price performance of Nvidia in the past month.

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested tool call (call_XyeNayvHXib38FAMsC1Mwz5l): task_planner *****
Arguments: 
{"question":"How can I summarize Nvidia's stock price performance over the past month and generate a blog post about it?"}
*****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION task_planner...
Call ID: call_XyeNayvHXib38FAMsC1Mwz5l
Input arguments: {'question': "How can I summarize Nvidia's stock price performance over the past month and generate a blog post about it?"}
planner_user (to planner):

How can I summarize Nvidia's stock price performance over the past mon

## Approach 2: Group chat

In [16]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
)


In [17]:
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Given a task, please determine what information is needed to complete the task.
Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code.
""",
    llm_config={"config_list": config_list, "cache_seed": None},
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Engineer. You write python/bash to retrieve relevant information. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You will write it for a task based on previous chat history. Don't write any code.""",
)

In [18]:
os.makedirs("paper", exist_ok=True)

In [19]:
code_executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    description="Executor should always be called after the engineer has written code to be executed.",
    human_input_mode="ALWAYS",
    code_execution_config={
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="paper"),
    },
)

In [20]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, code_executor, writer, planner],
    messages=[],
    max_round=20,
    speaker_selection_method="auto",
)

In [21]:
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list, "cache_seed": None})

In [22]:
# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Admin (to chat_manager):

Today is 2025-05-07. Write a blogpost about the stock price performance of Nvidia in the past month.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance over the past month (from April 7, 2025, to May 7, 2025), we will need the following information:

1. **Historical Stock Prices for Nvidia (NVDA)**  
   - Retrieve Nvidia's stock prices (daily data: open, high, low, close, and volume).
   - Data range: From April 7, 2025, to May 7, 2025.

2. **Percent Change in Stock Price**  
   - Calculate the percentage change in Nvidia's stock price over the period.
   - Identify how the stock opened and closed at the start (April 7, 2025) and end (May 7, 2025) of the period.

3. **Volatility**  
   - Assess the volatility of Nvidia's stock price using statistical metrics (e.g., standard deviation of daily price changes).

4. **Key E